# ===============================================================
# Boston Housing Price Prediction
# ===============================================================
# Goal: Predict median house values (MEDV) in Boston neighborhoods 
# and produce actionable insights.
# Key Tasks:
#   1. Explore and visualize the dataset (EDA).
#   2. Preprocess & handle missing values.
#   3. Train regularized models (Ridge, Lasso) and Polynomial+Ridge.
#   4. Perform feature selection (RFE) and explainability (permutation importance).
#   5. Compare models using cross-validation and evaluate on a held-out test set.
# Tools: Python, pandas, numpy, matplotlib, seaborn, scikit-learn, joblib
# Dataset: Kaggle (Boston Housing dataset)

In [ ]:
# ---------- IMPORT LIBRARIES ----------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import math

from pathlib import Path
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

In [ ]:
# ---------- FEATURE LABELS WITH UNITS ----------
feature_labels = {
    'CRIM':    'Crime Rate (per capita)',
    'ZN':      'Residential Land Zoned (%)',
    'INDUS':   'Industrial Land (%)',
    'CHAS':    'Charles River Dummy (0/1)',
    'NOX':     'Nitric Oxide Concentration (ppm)',
    'RM':      'Avg Rooms per Dwelling',
    'AGE':     'Pre-1940 Homes (%)',
    'DIS':     'Distance to Employment Centers (index)',
    'RAD':     'Highway Accessibility (index)',
    'TAX':     'Property Tax Rate (per $10k)',
    'PTRATIO': 'Pupil–Teacher Ratio',
    'B':       'Index of African American Population',
    'LSTAT':   'Lower Status Population (%)',
    'MEDV':    'Median House Value (1970 USD, $1000s)'
}

# ---------- PROJECT SETUP ----------
project_dir = Path.cwd()

fig_dir     = project_dir / "figures"
models_dir  = project_dir / "models"
outputs_dir = project_dir / "outputs"

for d in [fig_dir, models_dir, outputs_dir]:
    d.mkdir(exist_ok=True)

def save_plot(filename, width=8, height=5, dpi=300):
    """
    Save the current Matplotlib figure into the figures/ folder.
    """
    plt.gcf().set_size_inches(width, height)
    plt.savefig(fig_dir / filename, dpi=dpi, bbox_inches="tight")

def save_output(df, filename):
    """
    Save a pandas DataFrame into the outputs/ directory as CSV.
    """
    filepath = outputs_dir / filename
    df.to_csv(filepath, index=False)
    print(f"Saved output: {filepath}")
    return filepath

# -------------------- PLOT STYLE --------------------
sns.set(style="whitegrid")

In [ ]:
# ---------- LOAD DATA ----------
data_path = project_dir / "boston_housing_data.csv"

boston_df = pd.read_csv(
    data_path,
    header=None,
    sep=r"\s+"
)

# Column names
columns = [
    'CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS',
    'RAD','TAX','PTRATIO','B','LSTAT','MEDV'
]
boston_df.columns = columns

print(boston_df.head())
print(boston_df.isnull().sum())
print(boston_df.describe())

In [ ]:
# ---------- EXPLORATORY DATA ANALYSIS (EDA) ----------
# Histogram: Median House Values
plt.figure(figsize=(10, 6))
plt.hist(boston_df['MEDV'], bins=30, edgecolor='black')
plt.title('Distribution of Median House Values')
plt.xlabel(feature_labels['MEDV'])
plt.ylabel('Frequency')
save_plot("medv_distribution.png", width=10, height=6, dpi=300)
plt.show()

# Scatter Plot: Crime rate vs. Median House Value
plt.figure(figsize=(10, 6))
plt.scatter(boston_df['CRIM'], boston_df['MEDV'], alpha=0.5)
plt.title('Crime Rate vs. Median House Value')
plt.xlabel(feature_labels['CRIM'])
plt.ylabel(feature_labels['MEDV'])
save_plot("crim_vs_medv.png", width=10, height=6, dpi=300)
plt.show()

# Correlation Heatmap
corr = boston_df.corr()

plt.figure(figsize=(12, 10))
ax = sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlation Heatmap of Boston Housing Features")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
save_plot("correlation_heatmap.png", width=12, height=10, dpi=300)
plt.show()

# Boxplots of key features
features = ['CRIM', 'ZN', 'INDUS', 'RM', 'AGE', 'TAX', 'PTRATIO', 'LSTAT', 'MEDV']
n_cols = 3
n_features = len(features)
n_rows = math.ceil(n_features / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(features):
    sns.boxplot(y=boston_df[col], ax=axes[i])
    axes[i].set_title(f"Boxplot of {col}")
    axes[i].set_ylabel(feature_labels.get(col, col))

# Remove unused subplots if features don't fill the grid
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
save_plot("boxplots_all_features.png", width=15, height=n_rows * 4, dpi=300)
plt.show()

# Pairplots (selected features)
pair = sns.pairplot(
    boston_df,
    x_vars=['RM', 'LSTAT', 'PTRATIO'],
    y_vars='MEDV',
    height=4,
    kind='scatter'
)

# Add descriptive axis labels to pairplot
for ax in pair.axes.flat:
    xlabel = ax.get_xlabel()
    ylabel = ax.get_ylabel()
    if xlabel in feature_labels:
        ax.set_xlabel(feature_labels[xlabel])
    if ylabel in feature_labels:
        ax.set_ylabel(feature_labels[ylabel])

pair.savefig(
    fig_dir / "pairplots_selected.png",
    dpi=300,
    bbox_inches="tight"
)
plt.show()

In [ ]:
# ---------- PREPARE FEATURES AND TARGET ----------
X = boston_df.drop(columns=['MEDV'])
y = boston_df['MEDV'].copy()

# Train/test split (80/20)
TEST_SIZE = 0.2
RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, shuffle=True
)
print(f"Training rows: {X_train.shape[0]}, Test rows: {X_test.shape[0]}")

# ---------- PREPROCESSING PIPELINE ----------
numeric_features = X.columns.tolist()
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features)
], remainder='drop', verbose_feature_names_out=False)

# ---------- CROSS-VALIDATION FUNCTION ----------
def cv_rmse(estimator, X, y, cv=5):
    scores = cross_val_score(estimator, X, y, scoring='neg_root_mean_squared_error', cv=cv)
    return -scores.mean()

In [ ]:
# ---------- TRAIN MODELS ----------
# Ridge
ridge_pipe = Pipeline([
    ('preproc', preprocessor),
    ('ridge', Ridge(random_state=RANDOM_STATE))
])
ridge_grid = GridSearchCV(
    ridge_pipe,
    param_grid={'ridge__alpha': [0.1, 1.0, 10.0, 50.0, 100.0]},
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)
ridge_grid.fit(X_train, y_train)
best_ridge = ridge_grid.best_estimator_
ridge_cv_rmse = -ridge_grid.best_score_
print("Ridge best alpha:", ridge_grid.best_params_, "CV RMSE:", ridge_cv_rmse)

# Lasso
lasso_pipe = Pipeline([
    ('preproc', preprocessor),
    ('lasso', Lasso(max_iter=10000, random_state=RANDOM_STATE))
])
lasso_grid = GridSearchCV(
    lasso_pipe,
    param_grid={'lasso__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0]},
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)
lasso_grid.fit(X_train, y_train)
best_lasso = lasso_grid.best_estimator_
lasso_cv_rmse = -lasso_grid.best_score_
print("Lasso best alpha:", lasso_grid.best_params_, "CV RMSE:", lasso_cv_rmse)

# Polynomial Features + Ridge
poly_pipe = Pipeline([
    ('preproc', preprocessor),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('ridge', Ridge(random_state=RANDOM_STATE))
])
poly_grid = GridSearchCV(
    poly_pipe,
    param_grid={'ridge__alpha': [0.1, 1.0, 10.0]},
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)
poly_grid.fit(X_train, y_train)
best_poly = poly_grid.best_estimator_
poly_cv_rmse = -poly_grid.best_score_
print("Poly+Ridge best params:", poly_grid.best_params_, "CV RMSE:", poly_cv_rmse)

In [ ]:
# ---------- MODEL COMPARISON ----------
cv_summary = pd.DataFrame({
    'model': ['Ridge', 'Lasso', 'Poly+Ridge'],
    'cv_rmse': [ridge_cv_rmse, lasso_cv_rmse, poly_cv_rmse]
}).sort_values('cv_rmse').reset_index(drop=True)

print("\nCross-validated RMSE comparison:\n", cv_summary)

# Save CV summary to outputs
save_output(cv_summary, "cv_rmse_summary.csv")

plt.figure(figsize=(6, 4))
sns.barplot(data=cv_summary, x='model', y='cv_rmse')
plt.title("Cross-Validated RMSE by Model (Lower is Better)")
plt.ylabel("RMSE (MEDV; 1970 USD, $1000s)")
plt.xlabel("Model")
save_plot("cv_rmse_comparison.png", width=6, height=4, dpi=300)
plt.show()

# ---------- SELECT BEST MODEL ----------
best_model_name = cv_summary.loc[0, 'model']
best_model = {
    'Ridge': best_ridge,
    'Lasso': best_lasso,
    'Poly+Ridge': best_poly
}[best_model_name]
print("Selected best model based on CV:", best_model_name)

# Test set evaluation
y_pred_test = best_model.predict(X_test)
test_rmse = mean_squared_error(y_test, y_pred_test) ** 0.5
test_r2 = r2_score(y_test, y_pred_test)
print(f"Test RMSE ({best_model_name}): {test_rmse:.2f}")
print(f"Test R^2 ({best_model_name}): {test_r2:.2f}")

# Save test metrics
metrics_df = pd.DataFrame({
    "model": [best_model_name],
    "test_rmse": [test_rmse],
    "test_r2": [test_r2]
})
save_output(metrics_df, "test_metrics.csv")

# Save full test-set predictions
test_predictions = pd.DataFrame({
    "y_true": y_test.values,
    "y_pred": y_pred_test
})
save_output(test_predictions, "test_set_predictions.csv")

# Predicted vs True plot
plt.figure(figsize=(7, 5))
plt.scatter(y_pred_test, y_test, alpha=0.6)
plt.plot(
    [y_test.min(), y_test.max()],
    [y_test.min(), y_test.max()],
    'r--',
    lw=2
)
plt.xlabel(f"Predicted {feature_labels['MEDV']}")
plt.ylabel(f"True {feature_labels['MEDV']}")
plt.title(f"Predicted vs True (Test Set) — {best_model_name}")
save_plot("predicted_vs_true.png", width=7, height=5, dpi=300)
plt.show()

In [ ]:
# ---------- FEATURE SELECTION & EXPLAINABILITY ----------
# Preprocess train/test
X_train_pre = preprocessor.fit_transform(X_train)
X_test_pre = preprocessor.transform(X_test)

# RFE with LinearRegression
lr = LinearRegression()
rfe = RFE(estimator=lr, n_features_to_select=6)
rfe.fit(X_train_pre, y_train)
rfe_selected = [f for f, s in zip(numeric_features, rfe.get_support()) if s]
print("RFE selected top features (6):", rfe_selected)

rfe_df = pd.DataFrame({"selected_features": rfe_selected})
save_output(rfe_df, "rfe_selected_features.csv")

# Permutation importance
perm_res = permutation_importance(
    best_model,
    X_test,
    y_test,
    n_repeats=20,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
perm_df = pd.DataFrame({
    'feature': numeric_features,
    'importance_mean': perm_res.importances_mean,
    'importance_std': perm_res.importances_std
}).sort_values('importance_mean', ascending=False)
print("\nPermutation importance (top features):\n", perm_df.head(10))

save_output(perm_df, "permutation_importance.csv")

# Top permutation importance plot
plt.figure(figsize=(8, 5))
sns.barplot(
    data=perm_df.head(10),
    x='importance_mean',
    y='feature',
    palette='viridis',
    legend=False
)
plt.title("Permutation Importance (Test Set)")
plt.xlabel("Mean Decrease in Test Score")
plt.ylabel("Feature")
save_plot("permutation_importance.png", width=8, height=5, dpi=300)
plt.show()

# --- SAVE BEST MODEL ---
model_path = models_dir / "boston_best_model_refined.pkl"
joblib.dump(best_model, model_path)
print(f"\nSaved best model to: {model_path}")

In [ ]:
# ---------- DEMO PREDICTIONS ----------
demo_preds = best_model.predict(X_test.iloc[:5])
print("Demo predictions (first 5 rows of test set):", np.round(demo_preds, 2))
print("Actual values:", list(y_test.iloc[:5].values))

# ---------- PIPELINE COMPLETE ----------
print("Pipeline complete:")
print("- Data loading + EDA (distributions, correlations, pairplots)")
print("- Preprocessing (impute + scale) + train/test split")
print("- Model training (Ridge, Lasso, Poly+Ridge) with CV tuning")
print("- Model selection + test evaluation (RMSE, R^2)")
print("- Feature selection (RFE) + explainability (permutation importance)")
print("- Artifacts saved to /figures, /outputs, /models")